In [ ]:
# =========================
# Standard library
# =========================
import os
import sys
import time
import copy
import warnings
from typing import Optional, Tuple, Dict, Any

warnings.filterwarnings("ignore")

# =========================
# Core scientific stack
# =========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Jupyter-only (safe if executed in notebooks)
try:
    get_ipython().run_line_magic("matplotlib", "inline")
except Exception:
    pass

from tqdm import tqdm

# =========================
# PyTorch + TorchVision
# =========================
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision import transforms as T
import torchvision.models as tv_models

# =========================
# Medical imaging / model zoos
# =========================
from monai.networks.nets import DenseNet121

import timm
from timm.loss import LabelSmoothingCrossEntropy

# =========================
# Metrics (scikit-learn)
# =========================
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    cohen_kappa_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay,
)


In [ ]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [ ]:
def get_data_loaders(data_dir, batch_size, train = False):
    if train:
        #train
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(256),
         
            T.ToTensor(),
            T.Normalize(timm.data.IMAGENET_DEFAULT_MEAN, timm.data.IMAGENET_DEFAULT_STD), # imagenet means
            
        train_data = datasets.ImageFolder(os.path.join(data_dir, "train/"), transform = transform)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
        return train_loader, len(train_data)
    else:
        # val/test
        transform = T.Compose([ # We dont need augmentation for test transforms
            T.Resize(256),
            
            T.ToTensor(),
            T.Normalize(timm.data.IMAGENET_DEFAULT_MEAN, timm.data.IMAGENET_DEFAULT_STD), # imagenet means
        ])
        val_data = datasets.ImageFolder(os.path.join(data_dir, "validation/"), transform=transform)
        test_data = datasets.ImageFolder(os.path.join(data_dir, "test/"), transform=transform)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=0)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=0)
        return val_loader, test_loader, len(val_data), len(test_data)

In [ ]:
dataset_path = "new_directories"

In [ ]:
(train_loader, train_data_len) = get_data_loaders(dataset_path, 124, train=True)
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(dataset_path, 32, train=False)

In [ ]:
classes = get_classes("new_directories/train")
print(classes, len(classes))

In [ ]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

In [ ]:
print(len(train_loader), len(val_loader), len(test_loader))

In [ ]:
print(train_data_len, valid_data_len, test_data_len)

In [ ]:
# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)



In [ ]:
model = model.to(device)


In [ ]:

criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

In [ ]:
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10, device='cuda'):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Lists to store metrics per epoch
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-" * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            # Save metrics
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accs.append(epoch_acc)
            else:
                val_losses.append(epoch_loss)
                val_accs.append(epoch_acc)
            
            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))

    # Load best model weights
    model.load_state_dict(best_model_wts)

   
    
    return model, train_losses,val_losses, train_accs,val_accs


In [ ]:
model_ft,train_losses,val_losses, train_accs,val_accs = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=40) # now it is a lot faster


KeyboardInterrupt: 

In [ ]:
train_accs_on_cpu = [tensor.detach().cpu().numpy() for tensor in train_accs]
val_accs_on_cpu = [tensor.detach().cpu().numpy() for tensor in val_accs]

## Testing

In [ ]:
y_pred, y_true = [], []
model = model_ft.to(device)
with torch.no_grad():
    for x, y in tqdm(test_loader):
        pred = torch.argmax(model(x.to(device)), axis = 1).detach().cpu().numpy()
        y_pred.extend(pred)
        y_true.extend(y)
        

In [ ]:


print(classification_report(y_true, y_pred, target_names=classes, digits=4))

In [ ]:

cm = confusion_matrix(y_true, y_pred)

row_sums = cm.sum(axis=1, keepdims=True)

# Divide each element in the confusion matrix by the corresponding row sum and multiply by 100 to get percentages
confusion_percentages = np.round(cm / row_sums * 100,2)

disp = ConfusionMatrixDisplay(confusion_matrix=confusion_percentages,display_labels=labels)

disp.plot(cmap="Blues", values_format='')
plt.title("Resnet18", fontsize=16)
plt.show()

In [ ]:


finaltrain = pd.DataFrame([])

finaltrain = finaltrain._append({
                                        'Accuracy' : round(accuracy_score(y_true, y_pred)*100,3),
                                        'PrecisionTrain':round(precision_score(y_true, y_pred, average = 'weighted')*100,3),
                                        'RecallTrain':round(recall_score(y_true, y_pred, average = 'weighted')*100,3)  ,
                                        'F1Train':round(f1_score(y_true, y_pred, average = 'weighted')*100,3)}
                                      
                                        , ignore_index=True)
finaltrain 